In [ ]:
import os
import random

import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

### Basics

In [ ]:
SEED = 22
BATCH_SIZE = 2000
EPOCHS = 10
use_cuda = False

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

### Slightly EDA

In [ ]:
features = ['f'+str(i) for i in range(100)]

train_agg = train[features].agg(['max', 'min', 'mean', 'median']).transpose()
test_agg = test[features].agg(['max', 'min', 'mean', 'median']).transpose()

train_agg.plot(figsize=(30,10), grid=True, title='TRAIN DATA')
plt.show()

test_agg.plot(figsize=(30,10), grid=True, title='TEST DATA')
plt.show()

In [ ]:
# and without outliers

train_agg[train_agg["max"] < 100].plot(figsize=(30,10), grid=True, title='TRAIN DATA')
plt.show()

test_agg[test_agg["max"] < 100].plot(figsize=(30,10), grid=True, title='TEST DATA')
plt.show()

Looks like absolutely the same data distributions

### Prepare Dataloaders

In [ ]:
def preprocess_data(data, batch_size, num_workers=0, train=True):
    
    if train:
        
        val, train = data[:80000], data[80000:]
        
        train_target = torch.Tensor(train['target'].values)
        train_features = torch.Tensor(train.drop(['id', 'target'], axis=1).values)
        
        val_target = torch.Tensor(val['target'].values)
        val_features = torch.Tensor(val.drop(['id', 'target'], axis=1).values)

        train_loader = DataLoader(
            TensorDataset(train_features,train_target), 
            shuffle=True, 
            batch_size=batch_size,
            num_workers=num_workers
        )
        val_loader = DataLoader(
            TensorDataset(val_features,val_target), 
            shuffle=True, 
            batch_size=batch_size,
            num_workers=num_workers
        )
        
        return train_loader, val_loader
        
    test_features = torch.Tensor(data.drop(['id'], axis=1).values)
    
    test_loader = DataLoader(
        TensorDataset(test_features), 
        shuffle=False, 
        batch_size=batch_size,
        num_workers=num_workers
    )
    
    return test_loader

In [ ]:
train_loader, val_loader = preprocess_data(train, BATCH_SIZE, 0, True)
test_loader = preprocess_data(test, BATCH_SIZE, 0, False)

loaders = {
    'train': train_loader, 
    'valid': val_loader, 
    'test': test_loader
}

### Model

In [ ]:
criterion = nn.BCELoss()

def get_optimizer_scratch(model):
    return optim.Adam(model.parameters(), lr = 0.0015)

class EpicNet(nn.Module):
    def __init__(self):
        super(EpicNet, self).__init__()
        self.l1  = nn.Linear(100, 256)
        self.l2  = nn.Linear(256, 64)
        self.l3  = nn.Linear(64, 16)
        self.l4  = nn.Linear(16, 1)
        self.dropout = nn.Dropout(p=0.2)
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.batch_norm3 = nn.BatchNorm1d(16)

    def forward(self, x):
        x = self.l1(x)
        x = self.dropout(x)
        x = F.relu(self.l2(x))
        x = self.dropout(x)
        x = self.batch_norm2(x)
        x = F.relu(self.l3(x))
        x = self.dropout(x)
        x = self.batch_norm3(x)
        x = F.sigmoid(self.l4(x))

        return x

model = EpicNet()
print(model)

### Train

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):

            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += ((1/(batch_idx+1))*(loss.data.item()-train_loss))

        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target.unsqueeze(1))
                valid_loss += ((1/(1+batch_idx+1))*(loss.data.item()-valid_loss))

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    return model

In [ ]:
def custom_weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y,y)
        m.bias.data.fill_(0)
    
model.apply(custom_weight_init)
model = train(EPOCHS, loaders, model, get_optimizer_scratch(model), criterion, use_cuda, 'model_0.pt')

### Predict

In [ ]:
# upload final model
model.load_state_dict(torch.load('model_0.pt'))

In [ ]:
out = []
model.eval()

for batch_idx, data in enumerate(loaders['test']):
    out += model(data[0]).view(-1).tolist()

In [ ]:
predicts = pd.Series(out, name='target')

In [ ]:
predicts

In [ ]:
sub.drop(['target'], axis=1).join(predicts).to_csv('submission.csv', index=False)